In [1]:
from PIL import Image
import os
import numpy as np
import pandas as pd
import csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt

In [2]:
alex_images_path = "Alex_Kelly_Pics/Alex"
kelly_images_path = "Alex_Kelly_Pics/Kelly"
test_images_path = "Alex_Kelly_Pics/TestSet"
image_labels_path = "Kelly_and_Alex_Image_Labels - Sheet1.csv"

In [3]:
def sorted_image_names(path):
    return sorted(os.listdir(path), key=lambda x: int(''.join(filter(str.isdigit, x))))

In [4]:
alex_image_names = sorted_image_names(alex_images_path)
kelly_image_names = sorted_image_names(kelly_images_path)
test_image_names = sorted_image_names(test_images_path)

image_names = alex_image_names + kelly_image_names

In [5]:
labels = pd.read_csv(image_labels_path)
le = LabelEncoder().fit(labels["Landscape"])
labels["Landscape"] = le.transform(labels["Landscape"])
labels

,image_name,Human,Castle,Indoors,Landscape,Woman,Daytime,Children,Sunset,Flower,Animal,Building,Mask,Gray-Hair,Fire,Food_drink
0,Alex-Image01.png,1,0,0,3,0,1,1,0,0,1,0,0,0,0,0
1,Alex-Image02.png,1,0,0,3,0,1,1,0,0,0,1,0,0,0,0
2,Alex-Image03.png,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
3,Alex-Image04.png,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
4,Alex-Image05.png,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,Kelly-Image225.png,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0
481,Kelly-Image226.png,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0
482,Kelly-Image227.png,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0
483,Kelly-Image228.png,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0


In [6]:
labels['Photographer'] = labels['image_name'].str.split('-', n = 1).str[0]

In [7]:
le = LabelEncoder().fit(labels["Photographer"])
labels["Photographer"] = le.transform(labels["Photographer"])

In [8]:
labels

,image_name,Human,Castle,Indoors,Landscape,Woman,Daytime,Children,Sunset,Flower,Animal,Building,Mask,Gray-Hair,Fire,Food_drink,Photographer
0,Alex-Image01.png,1,0,0,3,0,1,1,0,0,1,0,0,0,0,0,0
1,Alex-Image02.png,1,0,0,3,0,1,1,0,0,0,1,0,0,0,0,0
2,Alex-Image03.png,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
3,Alex-Image04.png,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
4,Alex-Image05.png,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,Kelly-Image225.png,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,1
481,Kelly-Image226.png,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1
482,Kelly-Image227.png,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1
483,Kelly-Image228.png,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1


In [9]:
def get_largest_size(folder):
    biggest_image = [0, 0]
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename))
        if img.size[0] > biggest_image[0]: 
            biggest_image[0] = img.size[0]
        if img.size[1] > biggest_image[1]:
            biggest_image[1] = img.size[1]
    return biggest_image

In [10]:
alex_im_size = get_largest_size(alex_images_path)
kelly_im_size = get_largest_size(kelly_images_path)
test_im_size = get_largest_size(test_images_path)

print("Max Image Sizes:", max(alex_im_size, kelly_im_size, test_im_size))

Max Image Sizes: [667, 400]


In [11]:
def load_images_from_folder(folder, resize_shape=(700, 440)):
    images = []
    image_names = []
    count = 0
    for filename in os.listdir(folder):
        count += 1
        img = Image.open(os.path.join(folder, filename)).convert('RGB')
        img = img.resize(resize_shape)
        if img is not None:
            img_array = np.array(img)
            img_array = img_array.flatten()
            attr = labels[labels["image_name"] == filename].drop("image_name", axis = 1).to_numpy()
            img_array = np.append(img_array, attr)
            images.append(img_array)
            
        if count % int(len(os.listdir(folder)) / 10) == 0:
            print(f"{count / len(os.listdir(folder))*100}% Complete")
    return images

In [12]:
alex_images = load_images_from_folder(alex_images_path)
kelly_images = load_images_from_folder(kelly_images_path)
test_images = load_images_from_folder(test_images_path)

9.765625% Complete
19.53125% Complete
29.296875% Complete
39.0625% Complete
48.828125% Complete
58.59375% Complete
68.359375% Complete
78.125% Complete
87.890625% Complete
97.65625% Complete
9.606986899563319% Complete
19.213973799126638% Complete
28.82096069868996% Complete
38.427947598253276% Complete
48.03493449781659% Complete
57.64192139737992% Complete
67.24890829694323% Complete
76.85589519650655% Complete
86.46288209606988% Complete
96.06986899563319% Complete
10.0% Complete
20.0% Complete
30.0% Complete
40.0% Complete
50.0% Complete
60.0% Complete
70.0% Complete
80.0% Complete
90.0% Complete
100.0% Complete


In [13]:
merged_dataset = np.concatenate([alex_images, kelly_images])
del(alex_images, kelly_images) # Free up memory
merged_dataset

array([[153, 138, 137, ...,   0,   0,   0],
       [215, 206, 197, ...,   0,   0,   0],
       [103, 113, 121, ...,   0,   0,   0],
       ...,
       [ 82,  92,  78, ...,   0,   0,   1],
       [  5,   7,   6, ...,   0,   0,   1],
       [127, 124, 106, ...,   0,   0,   1]], dtype=int64)

In [14]:
# I am hesitant to turn this into a dataframe because of how much memory it consumes
# num_features = merged_dataset.shape[1]

# num_attributes = labels.shape[1] - 1

# columns = [f'pixel_{i}' for i in range(num_features - num_attributes)] + labels.drop("image_name", axis=1).columns.tolist()


# merged_df = pd.DataFrame(merged_dataset, columns=columns)

# merged_df

In [15]:
num_attributes = labels.shape[1] - 1

attribute_index = list(range(924000, 924000+num_attributes))

attributes = [col for col in labels.columns if col != "image_name"]

attr_dict = dict(zip(attributes, attribute_index))

In [16]:
# merged_df.to_csv('Complete_training_data.csv')


## CNN - Human

In [17]:
X = merged_dataset[:, 0:924000]
X = X / 255.0
y = merged_dataset[:, attr_dict["Human"]]

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train = X_train.reshape(X_train.shape[0],700,440,3)
X_test = X_test.reshape(X_test.shape[0],700,440,3)

annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0)
styles=[':','-.','--','-',':','-.','--','-',':','-.','--','-']

In [18]:
model = Sequential()
model.add(Conv2D(24,kernel_size=5,padding='same',activation='relu',
        input_shape=(700,440,3)))
model.add(MaxPool2D())
# if j>0:
#     model.add(Conv2D(48,kernel_size=5,padding='same',activation='relu'))
#     model.add(MaxPool2D())
# if j>1:
#     model.add(Conv2D(64,kernel_size=5,padding='same',activation='relu'))
#     model.add(MaxPool2D(padding='same'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

C:\Users\jamla\anaconda33\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
epochs = 20
history = model.fit(X_train, y_train, batch_size=32, epochs = epochs, 
validation_data = (X_test, y_test), callbacks=[annealer], verbose=2)
# print("CNN {0}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
#     names[j],epochs,max(history[j].history['accuracy']),max(history[j].history['val_accuracy'])))

Epoch 1/20


In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
axes = plt.gca()
axes.set_ylim([0.60,1])
plt.show()

# CNN - Sunset

In [ ]:
X = merged_dataset[:, 0:924000]
X = X / 255.0
y = merged_dataset[:, attr_dict[""]]

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train = X_train.reshape(X_train.shape[0],700,440,3)
X_test = X_test.reshape(X_test.shape[0],700,440,3)

annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0)
styles=[':','-.','--','-',':','-.','--','-',':','-.','--','-']